In [3]:
#IMPORTS NECESARIOS PARA EL PROYECTO
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import tqdm
import time
from sklearn import preprocessing
from torch.autograd import Variable
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, r2_score as r2
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import optim
from sklearn.metrics import r2_score
from gensim.models import TfidfModel
from gensim.matutils import corpus2dense
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from scipy.sparse import hstack
from nltk import download
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor  # Import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV  # Import GridSearchCV

nltk.download('punkt_tab')
nltk.download('punkt')
# Descargar los recursos necesarios
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\munoz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\munoz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\munoz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\munoz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\munoz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Carga del archivo JSON
fichero = "C:/Users/munoz/OneDrive/Desktop/UC3M/DATOS/full_format_recipes.json"
data = pd.read_json(fichero)

# Muestra las primeras filas
print(data.head())

                                          directions   fat  \
0  [1. Place the stock, lentils, celery, carrot, ...   7.0   
1  [Combine first 9 ingredients in heavy medium s...  23.0   
2  [In a large heavy saucepan cook diced fennel a...   7.0   
3  [Heat oil in heavy large skillet over medium-h...   NaN   
4  [Preheat oven to 350°F. Lightly grease 8x8x2-i...  32.0   

                       date  \
0 2006-09-01 04:00:00+00:00   
1 2004-08-20 04:00:00+00:00   
2 2004-08-20 04:00:00+00:00   
3 2009-03-27 04:00:00+00:00   
4 2004-08-20 04:00:00+00:00   

                                          categories  calories  \
0  [Sandwich, Bean, Fruit, Tomato, turkey, Vegeta...     426.0   
1  [Food Processor, Onion, Pork, Bake, Bastille D...     403.0   
2  [Soup/Stew, Dairy, Potato, Vegetable, Fennel, ...     165.0   
3  [Fish, Olive, Tomato, Sauté, Low Fat, Low Cal,...       NaN   
4  [Cheese, Dairy, Pasta, Vegetable, Side, Bake, ...     547.0   

                                           

In [5]:
############################ LIMPIO EL DATAFRAME #########################
data.replace(r'^\s*$', np.nan, regex=True, inplace=True)  # Reemplazar cadenas vacías por NaN
data_clean = data.dropna()
# Seleccionar solo las columnas numéricas
data_numeric = data_clean.select_dtypes(include=['number'])

data_cleaned = data_clean.copy()

exclude_column = 'rating'
# Iterar columna por columna y eliminar outliers usando IQR
for column in data_numeric.columns:
    if column == exclude_column:  
        print(f"Columna '{column}' ha sido excluida del proceso.")
        continue
    Q1 = data_cleaned[column].quantile(0.25)  # Primer cuartil
    Q3 = data_cleaned[column].quantile(0.75)  # Tercer cuartil
    IQR = Q3 - Q1                             # Rango intercuartílico

    # Definir los límites
    lower_bound = Q1 - 2 * IQR
    upper_bound = Q3 + 2 * IQR

    # Filtrar filas dentro de los límites para la columna actual
    data_cleaned = data_cleaned[
        (data_cleaned[column] >= lower_bound) & (data_cleaned[column] <= upper_bound)
    ]
    print(f"Columna '{column}': Límites [{lower_bound:.2f}, {upper_bound:.2f}]")
    print(f"Filas restantes después de limpiar '{column}': {data_cleaned.shape[0]}")

print("Tamaño original del dataset:", data_clean.shape)
print("Tamaño después de limpiar outliers:", data_cleaned.shape)

Columna 'fat': Límites [-43.00, 87.00]
Filas restantes después de limpiar 'fat': 10077
Columna 'calories': Límites [-520.00, 1280.00]
Filas restantes después de limpiar 'calories': 9917
Columna 'protein': Límites [-39.00, 66.00]
Filas restantes después de limpiar 'protein': 9584
Columna 'rating' ha sido excluida del proceso.
Columna 'sodium': Límites [-1051.00, 1784.00]
Filas restantes después de limpiar 'sodium': 9190
Tamaño original del dataset: (10608, 11)
Tamaño después de limpiar outliers: (9190, 11)


In [15]:
##################################### HUGGING FACE ##############################################


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import DatasetDict
from datasets import Dataset


texts = data_cleaned['desc']  # Usar 'desc' del dataframe limpio como entrada
labels = data_cleaned['rating']  # Usar 'rating' del dataframe limpio como salida

#Split de la entrada y salida en train y test
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

#BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)

#DATASET
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
test_dataset = Dataset(test_encodings, test_labels)


#ENTRENAMIENTO 
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=1,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps = 10,
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps=10,

)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

# EVALUACION
trainer.evaluate()
model.save_pretrained('./fine-tuned-bert')
tokenizer.save_pretrained('./fine-tuned-bert')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,13.165600
20,4.607500
30,1.220800
40,1.596800
50,1.495800
60,1.539200
70,1.615300
80,1.552300
90,1.516500
100,1.702500


Step,Training Loss
10,13.165600
20,4.607500
30,1.220800
40,1.596800
50,1.495800
60,1.539200
70,1.615300
80,1.552300
90,1.516500
100,1.702500


('./fine-tuned-bert/tokenizer_config.json',
 './fine-tuned-bert/special_tokens_map.json',
 './fine-tuned-bert/vocab.txt',
 './fine-tuned-bert/added_tokens.json')

In [1]:
import logging
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import DatasetDict
from datasets import Dataset


In [ ]:
###########################################  BERT ###############################################

from transformers import BertTokenizer, BertModel
import torch

# Cargar BERT

model = BertModel.from_pretrained('./fine-tuned-bert',output_hidden_states=True)
model.eval()

tokenizer = BertTokenizer.from_pretrained('./fine-tuned-bert')

def tokenize_BERT(text_list):

  embeddings = []
  for idx, text in enumerate(text_list):
    if isinstance(text, list):
            text = " ".join(text)  

    # Colocar tokens especiales de BERT
    marked_text = "[CLS] " + text + " [SEP]"

   
    tokenized_text = tokenizer.tokenize(marked_text)

    if len(tokenized_text) > 512:
            tokenized_text = tokenized_text[:512]  

    # Mapear los tokens a sus ids de BERT
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segment_id = idx % 2  # Diferenciar frases
    segments_ids = [segment_id] * len(tokenized_text)

    # Creamos tensores 
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensor = torch.tensor([segments_ids])

    # Generar embeddings  BERT
    with torch.no_grad():
      outputs = model(tokens_tensor, segments_tensor)
      hidden_states = outputs.last_hidden_state

    # Promediar los tokens para obtener el embedding del texto
    text_embedding = torch.mean(hidden_states, dim=1).squeeze().tolist()
    embeddings.append(text_embedding)

  return embeddings

direcciones = data_cleaned['directions'].tolist()
categorias = data_cleaned['categories'].tolist()
descripciones = data_cleaned['desc'].tolist()
titulo = data_cleaned['title'].tolist()






In [ ]:
directions_embeddings = tokenize_BERT(direcciones)

data_cleaned['directions_embeddings'] = directions_embeddings


In [8]:
categoriess_embeddings = tokenize_BERT(categorias)

data_cleaned['categoriess_embeddings'] = categoriess_embeddings


In [9]:
descriptions_embeddings = tokenize_BERT(descripciones)

data_cleaned['descriptions_embeddings'] = descriptions_embeddings


In [10]:

title_embeddings = tokenize_BERT(titulo)

data_cleaned['title_embeddings'] = title_embeddings

In [11]:
X_BERT = np.hstack([directions_embeddings, categoriess_embeddings, descriptions_embeddings, title_embeddings])

In [12]:
#Red Neuronal
class NN(nn.Module):
    def __init__(self, in_count):
        super(NN, self).__init__()
        self.fc =nn.Sequential(
            nn.Linear(in_count, 50),
            nn.ReLU(),
            nn.Linear(50, 25),
            nn.ReLU(),
            nn.Linear(25, 1)
        )

    def forward(self, x):
        return self.fc(x)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_BERT, data_cleaned['rating'], test_size=0.20, random_state=42)

Xtrain_tensor= torch.tensor(Xtrain, dtype=torch.float32)
Ytrain_tensor= torch.tensor(Ytrain.to_numpy(), dtype=torch.float32).unsqueeze(1)

Xtest_tensor= torch.tensor(Xtest, dtype=torch.float32)
Ytest_tensor= torch.tensor(Ytest.to_numpy(), dtype=torch.float32).unsqueeze(1)

BATCH_SIZE = 32

dataset_train = TensorDataset(Xtrain_tensor, Ytrain_tensor)
train_loader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)

model = NN(Xtrain.shape[1])

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

for epoch in range(25):
  for Xbatch, Ybatch in train_loader:
    optimizer.zero_grad()
    Ybatch_pred=model(Xbatch)
    loss = loss_fn(Ybatch_pred, Ybatch)
    loss.backward()
    optimizer.step()

model.eval()

with torch.no_grad():
  Ytest_pred=model(Xtest_tensor)
  loss_test = loss_fn(Ytest_pred, Ytest_tensor)
  print("Test Loss:", loss_test.item())
  Ytest_pred_array=Ytest_pred.numpy()
  valor_r2=r2(Ytest,Ytest_pred_array)
  print("R2:", valor_r2)

Test Loss: 1.5221141576766968
R2: 0.04408430935549723


In [13]:
#KNN

X_train, X_test, Y_train, Y_test = train_test_split(X_BERT, data_cleaned['rating'], test_size=0.30, random_state=42)
X_train_t, X_val, Y_train_t, Y_val = train_test_split(X_train, Y_train, test_size=0.30, random_state=42)

#Realizamos cross validation para encontrar el mejor valor del hiperparámetro k:
param_grid = {'n_neighbors': range(40, 70)}

knn_model = KNeighborsRegressor()
scorer = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = GridSearchCV(estimator=knn_model, param_grid=param_grid, scoring=scorer, cv=5, n_jobs=-1)
grid_search.fit(X_val, Y_val)
best_k = grid_search.best_params_['n_neighbors']
best_score = -grid_search.best_score_

print(f"Mejor valor de k: {best_k}")
print(f"MSE promedio para k={best_k}: {best_score}")

#realizo regresión:
k = best_k    # Number of neighbors
n_points = 1000

my_knn = KNeighborsRegressor(n_neighbors=k)
my_knn.fit(X_train_t,Y_train_t)

Y_predict = my_knn.predict(X_test)


MSE = mse(Y_test, Y_predict)
MAE = mae(Y_test, Y_predict)
RMSE = np.sqrt(MSE)
R2 = r2(Y_test, Y_predict)

print("MSE: ", MSE)
print("MAE: ", MAE)
print("RMSE: ", RMSE)
print("R2: ", R2)

Mejor valor de k: 57
MSE promedio para k=57: 1.587287980299239
MSE:  1.4826781664105122
MAE:  0.7781508313765916
RMSE:  1.2176527281661682
R2:  0.0394459174279439


In [14]:
#Random Forest

X_train, X_test, Y_train, Y_test = train_test_split(X_BERT, data_cleaned['rating'], test_size=0.30, random_state=42)
X_train_t, X_val, Y_train_t, Y_val = train_test_split(X_train, Y_train, test_size=0.30, random_state=42)

#Utilizamos hiperparámetros calculados antes

# Modelo 1: Random Forest
model_rf = RandomForestRegressor(n_estimators=500, max_depth = 10, random_state=42)
model_rf.fit(X_train_t, Y_train_t)

# Realizar predicciones
y_pred_rf = model_rf.predict(X_test)  # Predicciones con Random Forest


MSE = mse(Y_test, y_pred_rf)
MAE = mae(Y_test, y_pred_rf)
RMSE = np.sqrt(MSE)
R2 = r2(Y_test, y_pred_rf)

print("MSE: ", MSE)
print("MAE: ", MAE)
print("RMSE: ", RMSE)
print("R2: ", R2)

MSE:  1.3988340706618196
MAE:  0.7936792316451121
RMSE:  1.182723158926813
R2:  0.09376437324357423
